# Summarize Video

This notebook shows how to summarize a long piece of text representing a video transcript.

<a href="https://colab.research.google.com/github/atilatech/atila-core-service/blob/summarize_video/atlas/notebooks/summarize_video.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. We break up a long transcript into smaller chunks and summarize it using Facebook's BART model.

View [draft notebook with gpt-neox-20b and AllenAI long form](https://gist.github.com/ademidun/359c582c6f04780f3fb533ac5aaa832c#file-summarize_video-ipynb):

In [ ]:
%pip install transformers requests torch

In [ ]:
# Check that torch is available
# https://stackoverflow.com/a/48152675/5405197
import torch
print('torch.cuda.is_available()', torch.cuda.is_available())
print('torch.cuda.device_count()', torch.cuda.device_count())
print('torch.cuda.current_device()', torch.cuda.current_device())
print('torch.cuda.device(0)', torch.cuda.device(0))
print('torch.cuda.get_device_name(0)', torch.cuda.get_device_name(0))

In [48]:
from typing import Dict
from tqdm import tqdm
import time

from transformers import pipeline
import torch

class EndpointHandler():
    def __init__(self, path=""):
        
        # load the model
        SUMMARIZER_MODEL_NAME = "philschmid/bart-large-cnn-samsum"

        device = 0 if torch.cuda.is_available() else -1
        print(f'SUMMARIZER_MODEL_NAME will use: {device}. 0 = GPU -1 = CPU')

        t0 = time.time()
        self.summarizer = pipeline("summarization", model=SUMMARIZER_MODEL_NAME, device=device)
        t1 = time.time()

        total = t1 - t0
        print(f'Finished loading summarizer in {total} seconds')

    def __call__(self, data: Dict[str, str]) -> Dict:
        """
        Args:
            data (:obj:):
                includes the URL to video for transcription
        Return:
            A :obj:`dict`:. transcribed dict
        """
        # process input
        print('data', data)

        if "inputs" not in data:
            raise Exception(f"data is missing 'inputs' key which  EndpointHandler expects. Received: {data}"
                            f" See: https://huggingface.co/docs/inference-endpoints/guides/custom_handler#2-create-endpointhandler-cp")
        summarize = data.pop("summarize", False)

        if summarize:
            return {"summary": self.summarize_video(data["segments"])}
        else:
            return {"error": "no valid action found"}

    def summarize_video(self, segments):
        for index, segment in enumerate(segments):
            segment['summary'] = self.summarizer(segment['text'])
            segment['summary'] = segment['summary'][0]['summary_text']
            print('index', index)
            print('length', segment['length'])
            print('text', segment['text'])
            print('summary', segment['summary'])

        return segments
        

In [49]:
import requests # requests module of python (can be installed via pip)
import json
# https://www.youtube.com/watch?v=qaIBka_zHKI
# 20 OUTFIT IDEAS!! aka what to wear once we're out of quarantine lol
url = 'https://pastebin.com/raw/bTr2kMvw' # url of paste
r = requests.get(url) # response will be stored from url
outfit_ideas_video_segment = r.text  # raw text from url
# print(outfit_ideas_video_segment) # prints content
outfit_ideas_video_segment = json.loads(outfit_ideas_video_segment)
print(outfit_ideas_video_segment[0])


{'id': 'qaIBka_zHKI-t10', 'end': 28, 'url': 'https://www.youtube.com/watch?v=qaIBka_zHKI&t=10', 'text': "so I don't know about you guys but ever since this quarantine started I found myself scrolling through tick-tock and Pinterest seeing all these cute outfits maybe even catching myself online shopping a bit and it got me really inspired I wanted to see if I could put together some cute outfits with some", 'start': 10, 'title': "20 OUTFIT IDEAS!! aka what to wear once we're out of quarantine lol", 'views': 1447986, 'length': 825, 'duration': 18, 'video_id': 'qaIBka_zHKI', 'thumbnail': 'https://i.ytimg.com/vi/qaIBka_zHKI/sddefault.jpg'}


In [67]:
# By ChatGPT: https://imgur.com/a/ek9GI9S
def combine_segments(segments, max_length=1000):
    combined_segments = []
    if len(segments) < 1:
        return combined_segments

    current_segment = {"text": "", "end": 0}
    for index, segment in enumerate(segments):
        segment_keys = segment.keys()
        if len(current_segment["text"]) + len(segment["text"]) <= max_length:
            for key in segment_keys:
                if key not in current_segment:
                    current_segment[key] = segment[key]
            current_segment["text"] += segment["text"]
            current_segment["length"] = len(current_segment["text"])
            current_segment["duration"] += segment["duration"]
            current_segment["end"] = segment["end"]
        else:
            combined_segments.append(current_segment)
            current_segment = segment
    combined_segments.append(current_segment)
    return combined_segments

def get_evenly_spaced_elements(arr, elements_to_keep=10):
    if len(arr) <= elements_to_keep:
        return arr
    else:
        step = len(arr) // elements_to_keep
        evenly_spaced_elements = [arr[i] for i in range(0, len(arr), step) if i < len(arr)]
        while len(evenly_spaced_elements) < elements_to_keep and len(arr) > 0:
            evenly_spaced_elements.append(arr[-1])
        return evenly_spaced_elements



In [51]:
my_handler = EndpointHandler(path="")

SUMMARIZER_MODEL_NAME will use: 0. 0 = GPU -1 = CPU
Finished loading summarizer in 8.028823852539062 seconds


In [53]:
# prepare sample payload
combined_segments = get_evenly_spaced_elements(combine_segments(outfit_ideas_video_segment))
payload = {"summarize": True, "segments": combined_segments, "inputs": ""}


# # test the handler
payload_pred=my_handler(payload)
payload_pred

data {'summarize': True, 'segments': [{'text': "so I don't know about you guys but ever since this quarantine started I found myself scrolling through tick-tock and Pinterest seeing all these cute outfits maybe even catching myself online shopping a bit and it got me really inspired I wanted to see if I could put together some cute outfits with somepieces that I already own so I got up went through my closet tried on a few outfits and here's what I found hi guys welcome back to my channel so as you can probably tell by the title the intro and the thumbnail today's video is going to be 20 outfit ideas that you guys can wear at once we're out of quarantinewhich if you live in California like me that's probably not gonna be for a while but it's okay we're staying positive and by the time this is all over we're gonna have some really cute outfits to wear so I had this video idea awhile ago but I honestly did not think that I would be able to do it I thought I would be able", 'end': 58, 'id

Your max_length is set to 142, but you input_length is only 129. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=64)


index 1
length 939
text video and yeah I think that's about it I've been talking for too long so let's just get into the video okay so first of all is this outfit have this top references Holly and then this necklace from forever 21 these who have earrings also who princess Polly and then these jeans also forever 21 I really like this one I feellike this was really inspired by stuff I saw on Pinterest just warning you probably every single one of these outfits is going to be paired with airforces I know it's basic those are just kind of like the only cute shoes I have so we're working with it this outfit is just really comfy because of the big t-shirtand I like the way it looks with like some like ripped boyfriend jeans so that was out another one so here's outfit number two this one has to be one of my favorites I like the whole like monochromatic look I just think this one is so pretty I especially love the mini skirt because I've never had one of
summary The outfit inspired by Pinte

Your max_length is set to 142, but you input_length is only 132. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)


index 3
length 958
text from the last outfit again just the heart earrings from princess Polly and then the locket from my mom I've never seen a dress like this before it reminds me of like Tinker Bell or something this would actually be kind of a cute Tinker Bell costume if I like put my hair in a bun anyways I love this outfit it makes me feel really likeearly and I like have a look flowy two dresses I feel like a little kid so yeah that is outfit number five five I'm not sure alright here's the next outfit I'm honestly not too sure how I feel about this belt with this outfit but these jeans definitely need a belt these jeans are from forever 21 and then this littlelike button up is from Brandi and then for accessories I'm wearing this silver heart locket from Brandi these little silver heart earrings from Princess Holly and then this silver heart belt from Brandi it's such a minor detail but I love matching my accessories I feel like it looks so cute so when I have
summary The outfi

...

Your max_length is set to 142, but you input_length is only 131. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=65)


index 7
length 924
text gives me so much confidence I don't know why I just like that you can like cinch it up I don't know how to explain it like just the way that it's like cut I feel like it's very very flattering to your waistline I love the tie I feel like this top just has so much like flavor you know and then I decided topair it with this necklace from forever 21 I did something a little bit tricky with this necklace this is actually a layered necklace so if I move this to the front it looks really stupid but I decided to tuck it in in the back and my hair covers it because I didn't want like the gold chain this works muchbetter it's the shirt you're wearing doesn't have as much of like an open back so you can just have you this shirt and then I paired this with some pink star earring I think I showed these earlier in the video I just thought the pink and green looked really cool together I just feel like that color combination
summary The outfit that she is wearing is flatterin

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Your max_length is set to 142, but you input_length is only 133. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)


index 9
length 924
text I wore an outfit just like this on my birthday with a white top but personally I think the black top looks better because there's already like a lot in this skirt skirt is from forever 21 and then the top is actually a little moist hot food target that I just cut a little bit I've noticed that like somany of my clothes are boys clothes it's kind of weird and then I have this heart locket from brandy and then again the heart earrings from a princess Polly alright so next we have another really simple outfit so I just have this def leppard crop top from princess Holly these shorts again that are thrifted andthen they have like a little bit of like lip detail on them and then the red tag on the back so I thought it went well with the shirt so I'm wearing this necklace from forever 21 again which the gold chain and then with it since the necklace is like bigger I decided to go smaller with the earrings and I'm just
summary On her birthday she wore an outfit similar 

{'summary': [{'text': "so I don't know about you guys but ever since this quarantine started I found myself scrolling through tick-tock and Pinterest seeing all these cute outfits maybe even catching myself online shopping a bit and it got me really inspired I wanted to see if I could put together some cute outfits with somepieces that I already own so I got up went through my closet tried on a few outfits and here's what I found hi guys welcome back to my channel so as you can probably tell by the title the intro and the thumbnail today's video is going to be 20 outfit ideas that you guys can wear at once we're out of quarantinewhich if you live in California like me that's probably not gonna be for a while but it's okay we're staying positive and by the time this is all over we're gonna have some really cute outfits to wear so I had this video idea awhile ago but I honestly did not think that I would be able to do it I thought I would be able",
   'end': 58,
   'id': 'qaIBka_zHKI-t10

## Random Playground

The following is miscellaneous scripts I ran in the process of trying to get this to work.

In [70]:

elements = list(range(100))
evenly_spaced_elements = get_evenly_spaced_elements(elements, 7)
print(elements)
print(evenly_spaced_elements)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
[0, 14, 28, 42, 56, 70, 84, 98]


In [ ]:
import requests # requests module of python (can be installed via pip)
# https://www.youtube.com/watch?v=qaIBka_zHKI
# 20 OUTFIT IDEAS!! aka what to wear once we're out of quarantine lol
url = 'https://pastebin.com/raw/Qs0umEX0' # url of paste
r = requests.get(url) # response will be stored from url
outfit_ideas_video_transcript = r.text  # raw text from url
print(outfit_ideas_video_transcript) # prints content

In [41]:
combined_segments = combine_segments(outfit_ideas_video_segment)

print('outfit_ideas_video_segment', outfit_ideas_video_segment)
print('combined_segments', combined_segments)
print("combined_segments[0]", combined_segments[0])
print("combined_segments[1]", combined_segments[1])
print("combined_segments[2]", combined_segments[2])

outfit_ideas_video_segment [{'id': 'qaIBka_zHKI-t10', 'end': 44, 'url': 'https://www.youtube.com/watch?v=qaIBka_zHKI&t=10', 'text': "so I don't know about you guys but ever since this quarantine started I found myself scrolling through tick-tock and Pinterest seeing all these cute outfits maybe even catching myself online shopping a bit and it got me really inspired I wanted to see if I could put together some cute outfits with someso I don't know about you guys but ever since this quarantine started I found myself scrolling through tick-tock and Pinterest seeing all these cute outfits maybe even catching myself online shopping a bit and it got me really inspired I wanted to see if I could put together some cute outfits with somepieces that I already own so I got up went through my closet tried on a few outfits and here's what I found hi guys welcome back to my channel so as you can probably tell by the title the intro and the thumbnail today's video is going to be 20 outfit ideas that

combined_segments[0] {'id': 'qaIBka_zHKI-t10', 'end': 44, 'url': 'https://www.youtube.com/watch?v=qaIBka_zHKI&t=10', 'text': "so I don't know about you guys but ever since this quarantine started I found myself scrolling through tick-tock and Pinterest seeing all these cute outfits maybe even catching myself online shopping a bit and it got me really inspired I wanted to see if I could put together some cute outfits with someso I don't know about you guys but ever since this quarantine started I found myself scrolling through tick-tock and Pinterest seeing all these cute outfits maybe even catching myself online shopping a bit and it got me really inspired I wanted to see if I could put together some cute outfits with somepieces that I already own so I got up went through my closet tried on a few outfits and here's what I found hi guys welcome back to my channel so as you can probably tell by the title the intro and the thumbnail today's video is going to be 20 outfit ideas that you gu

In [14]:
from transformers import pipeline
import torch
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum", device=0 if torch.cuda.is_available() else -1)


# bart_summarizer = pipeline("summarization", model="facebook/bart-large-cnn",
#                            device=0 if torch.cuda.is_available() else -1)#device=0 means use GPU

In [ ]:

conversation = '''Jeff: Can I train a 🤗 Transformers model on Amazon SageMaker? 
Philipp: Sure you can use the new Hugging Face Deep Learning Container. 
Jeff: ok.
Jeff: and how can I get started? 
Jeff: where can I find documentation? 
Philipp: ok, ok you can find everything here. https://huggingface.co/blog/the-partnership-amazon-sagemaker-and-hugging-face                                           
'''
summarizer(conversation)

In [4]:
chunks = [outfit_ideas_video_transcript[i:i+1000] for i in range(0, len(outfit_ideas_video_transcript), 1000)]

In [47]:
# it's more efficient to do all the summarizations at once by passing an array to summarizer but that takes a long time
# Let's pass in a single string and print after each iteration.
for index, segment in enumerate(get_evenly_spaced_elements(combined_segments)):
    segment['summary'] = summarizer(segment['text'])[0]['summary_text']
    print('index', index)
    print('length', segment['length'])
    print('text', segment['text'])
    print('summary', segment['summary'])


combined_segments[0]

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


index 0
length 917
text so I don't know about you guys but ever since this quarantine started I found myself scrolling through tick-tock and Pinterest seeing all these cute outfits maybe even catching myself online shopping a bit and it got me really inspired I wanted to see if I could put together some cute outfits with someso I don't know about you guys but ever since this quarantine started I found myself scrolling through tick-tock and Pinterest seeing all these cute outfits maybe even catching myself online shopping a bit and it got me really inspired I wanted to see if I could put together some cute outfits with somepieces that I already own so I got up went through my closet tried on a few outfits and here's what I found hi guys welcome back to my channel so as you can probably tell by the title the intro and the thumbnail today's video is going to be 20 outfit ideas that you guys can wear at once we're out of quarantine
summary There are 20 outfit ideas that you guys can wear a

Your max_length is set to 142, but you input_length is only 69. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)


index 1
length 944
text know I have so much stuff that I could donate and you guys have also been really wanting me to sell stuff on depop so I'll definitely think about that but if you guys want to see my closet clean out like if you guys want me to do a video on it I can definitely do that just comment down below if you want thatvideo and yeah I think that's about it I've been talking for too long so let's just get into the video okay so first of all is this outfit have this top references Holly and then this necklace from forever 21 these who have earrings also who princess Polly and then these jeans also forever 21 I really like this one I feellike this was really inspired by stuff I saw on Pinterest just warning you probably every single one of these outfits is going to be paired with airforces I know it's basic those are just kind of like the only cute shoes I have so we're working with it this outfit is just really comfy because of the big t-shirt
summary I have a lot of stuff t

Your max_length is set to 142, but you input_length is only 63. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=31)


...

Your max_length is set to 142, but you input_length is only 68. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=34)


index 7
length 923
text better it's the shirt you're wearing doesn't have as much of like an open back so you can just have you this shirt and then I paired this with some pink star earring I think I showed these earlier in the video I just thought the pink and green looked really cool together I just feel like that color combinationjust so cute I reminds me of like Cosmo and Wanda but it's okay another thing when I could do I could wear this cute purse with it I'm princess Polly oh I love this oh my gosh I'm like getting excited to wear this it also matches the earrings which i think is really cute I feel like some people would love thisand then I feel like some people would hate it just depends if you like the colors pink and green together personally I love them obviously the first is from princess Polly the shirt is also from princess Polly the skirt is from Tilly's the earrings are from that one website like a choker or whatever
summary The shirt is from princess Polly, the skirt 

Your max_length is set to 142, but you input_length is only 40. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=20)


index 10
length 929
text but the jeans are like a tiny bit like almost flared at the bottom which i think is really cute with this and then these jeans are also super high-waisted so I thought it would be perfect to wear with this top this top is the tiniest bit out of my comfort zone just because of like the open stomach right here butI still really like it I'm wearing these jeans from a cotton on and then the top is from princess Polly and then I'm wearing these really cool star hoop earrings from forever 21 I believe I love these earrings I think they look really cool with this outfit so that is all of the outfits that I have for youguys I really hope you guys liked this video there was so much effort that went into it there was two days of filming a lot of preparation putting together all the outfits and I don't even know how long it's gonna take to edit probably a very long time if you watched my last video you probably know that I was
summary In the video, she is wearing jeans fr

{'text': "so I don't know about you guys but ever since this quarantine started I found myself scrolling through tick-tock and Pinterest seeing all these cute outfits maybe even catching myself online shopping a bit and it got me really inspired I wanted to see if I could put together some cute outfits with someso I don't know about you guys but ever since this quarantine started I found myself scrolling through tick-tock and Pinterest seeing all these cute outfits maybe even catching myself online shopping a bit and it got me really inspired I wanted to see if I could put together some cute outfits with somepieces that I already own so I got up went through my closet tried on a few outfits and here's what I found hi guys welcome back to my channel so as you can probably tell by the title the intro and the thumbnail today's video is going to be 20 outfit ideas that you guys can wear at once we're out of quarantine",
 'end': 44,
 'id': 'qaIBka_zHKI-t10',
 'url': 'https://www.youtube.com